<a href="https://colab.research.google.com/github/GarciaLnk/LlamaRec/blob/main/demo_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# LlamaRec Demo Notebook

This notebook is provided to allow users to deploy the LlamaRec demo on a free GPU-enabled provider like Google Colab.


## Install mamba on Colab (run individually)


In [ ]:
try:
    import google.colab
    %pip install -q condacolab
    import condacolab
    condacolab.install()
except Exception:
    pass

## Download demo files


In [ ]:
!wget https://files.garcialnk.com/llamarec-demo.zip -O ./demo.zip
!unzip ./demo.zip
%rm ./demo.zip

## Install dependencies


In [ ]:
!mamba install -y streamlit pytorch pytorch-cuda=12.1 transformers peft filelock whoosh -c pytorch -c nvidia
%pip install --no-deps bitsandbytes triton
%pip install flash-attn --no-build-isolation

## Forward port 8501


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x ./cloudflared-linux-amd64

import subprocess
import threading

def run_process(cmd):
    with open('output.txt', 'w') as f:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        if process.stdout:
            for line in iter(process.stdout.readline, b''):
                f.write(line.decode())
                f.flush()

cmd = ["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"]
thread = threading.Thread(target=run_process, args=(cmd,))
thread.start()

!sleep 5
!grep -o 'https://.*\.trycloudflare.com' output.txt | head -n 1 | xargs -I {} echo "Your tunnel url {}"

## Start demo


In [ ]:
%cd ./demo
!streamlit run ./app.py --server.address=localhost --server.port 8501